In [ ]:
import threading
import os
import ctypes
import numpy as np
from picosdk.ps4000a import ps4000a as ps
import matplotlib.pyplot as plt
from picosdk.functions import adc2mV, assert_pico_ok
import time
import pandas as pd
from scipy.signal import butter, lfilter, freqz, medfilt, hann, convolve, welch, get_window, tukey, gaussian, hilbert, iirfilter
from scipy.misc import electrocardiogram
from scipy.signal import find_peaks
import random
import itertools
import datetime
import ipywidgets as widgets
from scipy.optimize import minimize
from filterpy.monte_carlo import systematic_resample
from numpy.linalg import norm
from numpy.random import randn
import scipy.stats
from scipy.spatial.distance import cdist
import ipympl as ipy
import math
from matplotlib.patches import Rectangle
from datetime import datetime
import serial


In [ ]:
                                          #--------------Global variables--------------#
channelA = np.array([])
channelB = np.array([])
channelC = np.array([])
channelD = np.array([])
channelE = np.array([])

Ox =  np.array([])
Oy =  np.array([])
Oz =  np.array([])
Ax =  np.array([])
Ay =  np.array([])
Az =  np.array([])

T = 0

In [ ]:
def gps_solve(distances_to_station, stations_coordinates):
    def error(x, c, r):
        return sum([(np.linalg.norm(x - c[i]) - r[i]) ** 2 for i in range(len(c))])

    l = len(stations_coordinates)
    S = sum(distances_to_station)
    # compute weight vector for initial guess
    W = [((l - 1) * S) / (S - w) for w in distances_to_station]
    # get initial guess of point location
    x0 = sum([W[i] * stations_coordinates[i] for i in range(l)])
    # optimize distance from signal origin to border of spheres
    return minimize(error, x0, args=(stations_coordinates, distances_to_station), method='Nelder-Mead').x

# Pinger co-ordinates = [1 ,1, 1]
# Location of hdphones = [1.70,0.0, 0.75] , [0.80, 8.85, 0.25], [0.0, 4.25, 0.55], [3.0, 4.50, 1.20]
# Radius of spheres = [1.2459, 7.8882, 3.4300, 4.0360] #respectively

    landmarks = np.array([[1.70, 0.0, 0.75], [0.80, 8.85, 0.25], [0.0, 4.25, 0.55], [3.0, 4.50, 1.20]])

    H1 = landmarks[0]
    H2 = landmarks[1]
    H3 = landmarks[2]
    H4 = landmarks[3]

    d1 = dist_AB
    d2 = dist_AC
    d3 = dist_AD
    d4 = dist_AE

    if __name__ == "__main__":
        stations = list(np.array([H1, H2, H3, H4]))
        distances_to_station = [d1, d2, d3, d4]
        print('\n')
        print(gps_solve(distances_to_station, stations))
        print('\033[95m' + 'Co-ordinates of the pinger = [ 0.0  0.65  1.0 ]' + '\033[95m')
        print ('\033[0m')
        pinger_coordinates_multi = gps_solve(distances_to_station, stations)
        
    return pinger_coordinates_multi

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = iirfilter(order, [low, high],
                        btype='bandpass', ftype='butter')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    signal = lfilter(b, a, data)
    return signal

# definition for signal normalisation in range -1 to +1
def normalisation(signal):
    signal = signal - signal.mean()
    signal = signal / signal.max()
    return signal

def filter_bandpass(signal, fs):
    lowcut = 43e3
    highcut = 47e3
    signal = np.array(signal)
    signal = butter_bandpass_filter(signal, lowcut, highcut, fs, order=6)
    signal_p = normalisation(signal)
    return signal_p

In [ ]:
from numpy.random import uniform


def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 5))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    particles[:, 3] = mean[3] + (randn(N) * std[3])
    particles[:, 3] %= 2 * np.pi
    particles[:, 4] = mean[4] + (randn(N) * std[4])
    particles[:, 4] %= 2 * np.pi
    return particles


def create_uniform_particles(x_range, y_range, z_range, xy_heading, z_heading, N):
    particles = np.empty((N, 5))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)                 #-----X-----#
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)                 #-----Y-----#
    particles[:, 2] = uniform(z_range[0], z_range[1], size=N)                 #-----Z-----#
    particles[:, 3] = uniform(xy_heading[0], xy_heading[1], size=N)
    particles[:, 3] = particles[:, 3] % 2 * np.pi                             #-----heading in XY plane wrt X axis-----#
    particles[:, 4] = uniform(z_heading[0], z_heading[1], size=N)
    particles[:, 4] = particles[:, 4] % 2 * np.pi                             #-----heading wrt Z axis-----#
    return particles



In [ ]:
def predict(particles, u, std, dt=1.):

    N = len(particles)
    # update heading
    particles[:, 3] += u[0] 
    particles[:, 3] %= 2 * np.pi
    particles[:, 4] = particles[:, 4] + u[1]
    particles[:, 4] %= 2 * np.pi
    
    # move in the (noisy) commanded direction
    
    particles[:, 0] = (particles[:, 0]) + ( u[2] * math.sin(u[1]) * math. cos(u[0]) )
    particles[:, 1] = (particles[:, 1]) + ( u[2] * math.sin(u[1]) * math. sin(u[0]) )
    particles[:, 2] = (particles[:, 2]) + ( u[2] * math.cos(u[1]) )  

In [ ]:
def correct(N, particles, pinger_coordinates_multi):                     #---------------Not in use---------------#
    thetaX = np.empty((N))
    distance = np.linalg.norm(particles[:, 0:3] - pinger_coordinates_multi, axis=1)
    pinger_coordinates_multi = pinger_coordinates_multi.astype('float64')
    theta_X = np.arctan((pinger_coordinates_multi[1]-particles[:, 1])/(pinger_coordinates_multi[0]- particles[:, 0]))
    
    a = pinger_coordinates_multi[0]-particles[:, 0]
    b = pinger_coordinates_multi[1]-particles[:, 1]
    c = pinger_coordinates_multi[2]-particles[:, 2]
    mod = np.abs(np.sqrt(a**2 +b**2 +c**2))
    theta_Z = np.arccos(c/(mod))
    
    
    particles[:, 0] = (particles[:, 0]) + ( 0.25*distance[:] * np.cos(theta_X[:]))
    particles[:, 1] = (particles[:, 1]) + ( 0.25*distance[:] * np.sin(theta_X[:]))
    particles[:, 2] = (particles[:, 2]) + ( 0.25*distance[:] * np.sin(theta_Z[:]))

In [ ]:
def update(particles, weights, z, R, landmarks):                                      
    for i, landmark in enumerate(landmarks):
        distance = np.linalg.norm(particles[:, 0:3] - landmark, axis=1)   
        weights *= scipy.stats.norm(distance, R).pdf(z[i])

    weights += 1.e-300      # avoid round-off to zero
    weights /= sum(weights) # normalize       

In [ ]:
def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""

    pos = particles[:, 0:5]                                              
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var

def neff(weights):                                                                     #------Unchanged cell------#
    return 1. / np.sum(np.square(weights))


def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights.resize(len(particles))
    weights.fill (1.0 / len(weights))

In [ ]:
# Create chandle and status ready for use
chandle = ctypes.c_int16()
status = {}

# Open PicoScope 4000 Series device
# Returns handle to chandle for use in future API functions
status["openunit"] = ps.ps4000aOpenUnit(ctypes.byref(chandle), None)


try:
    assert_pico_ok(status["openunit"])
    print('open unit done')
except:

    powerStatus = status["openunit"]

    if powerStatus == 286:
        status["changePowerSource"] = ps.ps4000aChangePowerSource(chandle, powerStatus)
    else:
        raise

    assert_pico_ok(status["changePowerSource"])


enabled = 1
disabled = 0
analogue_offset = 0.0

# Set up channel A
# handle = chandle
# channel = PS4000A_CHANNEL_A = 0
# enabled = 1
# coupling type = PS4000A_DC = 1
# range = PS4000A_2V = 7
# analogue offset = 0 V
channel_range = 7
status["setChA"] = ps.ps4000aSetChannel(chandle,
                                        ps.PS4000A_CHANNEL['PS4000A_CHANNEL_A'],
                                        enabled,
                                        ps.PS4000A_COUPLING['PS4000A_DC'],
                                        channel_range,
                                        analogue_offset)
assert_pico_ok(status["setChA"])

# Set up channel B
# handle = chandle
# channel = PS4000A_CHANNEL_B = 0
# enabled = 1
# coupling type = PS4000A_DC = 1
# range = PS4000A_2V = 7
# analogue offset = 0 V
channel_range = 7
status["setChB"] = ps.ps4000aSetChannel(chandle,
                                        ps.PS4000A_CHANNEL['PS4000A_CHANNEL_B'],
                                        enabled,
                                        ps.PS4000A_COUPLING['PS4000A_DC'],
                                        channel_range,
                                        analogue_offset)
assert_pico_ok(status["setChB"])

# Set up channel C
# handle = chandle
# channel = PS4000A_CHANNEL_C = 2
# enabled = 1
# coupling type = PS4000A_DC = 1
# range = PS4000A_2V = 7
# analogue offset = 0 V
channel_range = 7
status["setChC"] = ps.ps4000aSetChannel(chandle,
                                        ps.PS4000A_CHANNEL['PS4000A_CHANNEL_C'],
                                        enabled,
                                        ps.PS4000A_COUPLING['PS4000A_DC'],
                                        channel_range,
                                        analogue_offset)
assert_pico_ok(status["setChC"])


# Set up channel D
# handle = chandle
# channel = PS4000A_CHANNEL_D = 3
# enabled = 1
# coupling type = PS4000A_DC = 1
# range = PS4000A_2V = 7
# analogue offset = 0 V
channel_range = 7
status["setChD"] = ps.ps4000aSetChannel(chandle,
                                        ps.PS4000A_CHANNEL['PS4000A_CHANNEL_D'],
                                        enabled,
                                        ps.PS4000A_COUPLING['PS4000A_DC'],
                                        channel_range,
                                        analogue_offset)
assert_pico_ok(status["setChD"])


# Set up channel E
# handle = chandle
# channel = PS4000A_CHANNEL_A = 4
# enabled = 1
# coupling type = PS4000A_DC = 1
# range = PS4000A_2V = 7
# analogue offset = 0 V
channel_range = 7
status["setChE"] = ps.ps4000aSetChannel(chandle,
                                        ps.PS4000A_CHANNEL['PS4000A_CHANNEL_E'],
                                        enabled,
                                        ps.PS4000A_COUPLING['PS4000A_DC'],
                                        channel_range,
                                        analogue_offset)
assert_pico_ok(status["setChE"])

# Size of capture, repetation rate should be 0.5 seconds.
sizeOfOneBuffer = 10000
numBuffersToCapture = 50                                                       
# Capturing for 1 Sec with 1000*1000, for 0.5 sec divide one of them by 2
totalSamples = sizeOfOneBuffer * numBuffersToCapture

# Create buffers ready for assigning pointers for data collection
bufferAMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferBMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferCMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferDMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)
bufferEMax = np.zeros(shape=sizeOfOneBuffer, dtype=np.int16)

memory_segment = 0


def streaming_callback(handle, noOfSamples, startIndex, overflow, triggerAt, triggered, autoStop, param):
    global nextSample, autoStopOuter, wasCalledBack
    wasCalledBack = True
    destEnd = nextSample + noOfSamples
    sourceEnd = startIndex + noOfSamples
    bufferCompleteA[nextSample:destEnd] = bufferAMax[startIndex:sourceEnd]
    bufferCompleteB[nextSample:destEnd] = bufferBMax[startIndex:sourceEnd]
    bufferCompleteC[nextSample:destEnd] = bufferCMax[startIndex:sourceEnd]
    bufferCompleteD[nextSample:destEnd] = bufferDMax[startIndex:sourceEnd]
    bufferCompleteE[nextSample:destEnd] = bufferEMax[startIndex:sourceEnd]
    nextSample += noOfSamples
    if autoStop:
        autoStopOuter = True

In [ ]:
for itr in range(0, 5):
    print('iteration number: ', itr+1)


    def getArduinoData():
        global Ox, Oy, Oz, Ax, Ay, Az, T
        arduinoData=serial.Serial('com3',115200)
        time.sleep(0.5)
        n=0
        g = [0.0, 0.0, 0.0]

        while (n<950):
            while (arduinoData.inWaiting()==0):
                pass

            dataPacket=arduinoData.readline()

            dataPacket=str(dataPacket,'utf-8')
            splitPacket=dataPacket.split(',')




            dateTimeObj = datetime.now()
            A_x=float(splitPacket[0])
            A_y=float(splitPacket[1])
            A_z=float(splitPacket[2])
            Pitch=float(splitPacket[3])
            Roll=float(splitPacket[4])
            Yaw=float(splitPacket[5])
            q0=float(splitPacket[6])
            q1=float(splitPacket[7])
            q2=float(splitPacket[8])
            q3=float(splitPacket[9])
            g[0] = 2 * (q1 * q3 - q0 * q2)
            g[1] = 2 * (q0 * q1 + q2 * q3)
            g[2] = q0 * q0 - q1 * q1 - q2 * q2 + q3 * q3
            Acc_x = A_x - g[0]
            Acc_y = A_y - g[1]
            Acc_z = A_z - g[2]
    #         print("             HH:SS:MS    :     ", dateTimeObj.minute, ':', dateTimeObj.second, ':', dateTimeObj.microsecond)
    #         print("AccX=",A_x, "             AccY = ",A_y, "        AccZ = ",A_z)
    #         print("Pitch = ",Pitch,"        Roll = ",Roll,"       Yaw = ",Yaw,"\n\n")



            Ax = np.append (Ax, Acc_x)                    #-------------row-------------#
            Ay = np.append (Ay, Acc_y)                    #-------------row-------------#
            Az = np.append (Az, Acc_z)                    #-------------row-------------#

            Ox = np.append (Ox, Pitch)                  #-------------row-------------#
            Oy = np.append (Oy, Roll)                   #-------------row-------------#
            Oz = np.append (Oz, Yaw)                    #-------------row-------------#


            n = n+1      
        DOx = (Ox[n-1]-Ox[0])
        DOy = (Oy[n-1]-Oy[0])
        DOz = (Oz[n-1]-Oz[0])

        DAx = (Ox[n-1]-Ox[0])
        DAy = (Oy[n-1]-Oy[0])
        DAz = (Oz[n-1]-Oz[0])


        return [Ox, Oy, Oz, Ax, Ay, Az]


    def getHPdata():
        global nextSample, channelA, channelB, channelC, channelD, channelE, T
            # Set data buffer location for data collection from channel A
        # handle = chandle
        # source = PS4000A_CHANNEL_A = 0
        # pointer to buffer max = ctypes.byref(bufferAMax)
        # pointer to buffer min = ctypes.byref(bufferAMin)
        # buffer length = maxSamples
        # segment index = 0
        # ratio mode = PS4000A_RATIO_MODE_NONE = 0
        status["setDataBuffersA"] = ps.ps4000aSetDataBuffers(chandle,
                                                            ps.PS4000A_CHANNEL['PS4000A_CHANNEL_A'],
                                                            bufferAMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                            None,
                                                            sizeOfOneBuffer,
                                                            memory_segment,
                                                            ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'])
        assert_pico_ok(status["setDataBuffersA"])

        # Set data buffer location for data collection from channel B
        # handle = chandle
        # source = PS4000A_CHANNEL_B = 1
        # pointer to buffer max = ctypes.byref(bufferBMax)
        # pointer to buffer min = ctypes.byref(bufferBMin)
        # buffer length = maxSamples
        # segment index = 0
        # ratio mode = PS4000A_RATIO_MODE_NONE = 0
        status["setDataBuffersB"] = ps.ps4000aSetDataBuffers(chandle,
                                                            ps.PS4000A_CHANNEL['PS4000A_CHANNEL_B'],
                                                            bufferBMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                            None,
                                                            sizeOfOneBuffer,
                                                            memory_segment,
                                                            ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'])
        assert_pico_ok(status["setDataBuffersB"])


        # Set data buffer location for data collection from channel C
        # handle = chandle
        # source = PS4000A_CHANNEL_C = 2
        # pointer to buffer max = ctypes.byref(bufferCMax)
        # pointer to buffer min = ctypes.byref(bufferDMin)
        # buffer length = maxSamples
        # segment index = 0
        # ratio mode = PS4000A_RATIO_MODE_NONE = 0
        status["setDataBuffersC"] = ps.ps4000aSetDataBuffers(chandle,
                                                            ps.PS4000A_CHANNEL['PS4000A_CHANNEL_C'],
                                                            bufferCMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                            None,
                                                            sizeOfOneBuffer,
                                                            memory_segment,
                                                            ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'])
        assert_pico_ok(status["setDataBuffersC"])

        # Set data buffer location for data collection from channel D
        # handle = chandle
        # source = PS4000A_CHANNEL_D = 3
        # pointer to buffer max = ctypes.byref(bufferDMax)
        # pointer to buffer min = ctypes.byref(bufferDMin)
        # buffer length = maxSamples
        # segment index = 0
        # ratio mode = PS4000A_RATIO_MODE_NONE = 0
        status["setDataBuffersD"] = ps.ps4000aSetDataBuffers(chandle,
                                                            ps.PS4000A_CHANNEL['PS4000A_CHANNEL_D'],
                                                            bufferDMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                            None,
                                                            sizeOfOneBuffer,
                                                            memory_segment,
                                                            ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'])
        assert_pico_ok(status["setDataBuffersD"])


        # Set data buffer location for data collection from channel E
        # handle = chandle
        # source = PS4000A_CHANNEL_E = 4
        # pointer to buffer max = ctypes.byref(bufferEMax)
        # pointer to buffer min = ctypes.byref(bufferEMin)
        # buffer length = maxSamples
        # segment index = 0
        # ratio mode = PS4000A_RATIO_MODE_NONE = 0


        status["setDataBuffersE"] = ps.ps4000aSetDataBuffers(chandle,
                                                            ps.PS4000A_CHANNEL['PS4000A_CHANNEL_E'],
                                                            bufferEMax.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                            None,
                                                            sizeOfOneBuffer,
                                                            memory_segment,
                                                            ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'])
        assert_pico_ok(status["setDataBuffersE"])



        # Begin streaming mode:
        sampleInterval = ctypes.c_int32(10)
        sampleUnits = ps.PS4000A_TIME_UNITS['PS4000A_US']
        # We are not triggering:
        maxPreTriggerSamples = 0
        autoStopOn = 1
        # No downsampling:
        downsampleRatio = 1


        status["runStreaming"] = ps.ps4000aRunStreaming(chandle,
                                                        ctypes.byref(sampleInterval),
                                                        sampleUnits,
                                                        maxPreTriggerSamples,
                                                        totalSamples,
                                                        autoStopOn,
                                                        downsampleRatio,
                                                        ps.PS4000A_RATIO_MODE['PS4000A_RATIO_MODE_NONE'],
                                                        sizeOfOneBuffer)
        assert_pico_ok(status["runStreaming"])


        actualSampleInterval = sampleInterval.value
        actualSampleIntervalNs = actualSampleInterval * 1000

        print("Capturing at sample interval %s us" % actualSampleInterval)


        # We need a big buffer, not registered with the driver, to keep our complete capture in.
        bufferCompleteA = np.zeros(shape=totalSamples, dtype=np.int16)
        bufferCompleteB = np.zeros(shape=totalSamples, dtype=np.int16)
        bufferCompleteC = np.zeros(shape=totalSamples, dtype=np.int16)
        bufferCompleteD = np.zeros(shape=totalSamples, dtype=np.int16)
        bufferCompleteE = np.zeros(shape=totalSamples, dtype=np.int16)
        nextSample = 0
        autoStopOuter = False
        wasCalledBack = False


        def streaming_callback(handle, noOfSamples, startIndex, overflow, triggerAt, triggered, autoStop, param):
            global nextSample, autoStopOuter, wasCalledBack
            wasCalledBack = True
            destEnd = nextSample + noOfSamples
            sourceEnd = startIndex + noOfSamples
            bufferCompleteA[nextSample:destEnd] = bufferAMax[startIndex:sourceEnd]
            bufferCompleteB[nextSample:destEnd] = bufferBMax[startIndex:sourceEnd]
            bufferCompleteC[nextSample:destEnd] = bufferCMax[startIndex:sourceEnd]
            bufferCompleteD[nextSample:destEnd] = bufferDMax[startIndex:sourceEnd]
            bufferCompleteE[nextSample:destEnd] = bufferEMax[startIndex:sourceEnd]
            nextSample += noOfSamples
            if autoStop:
                autoStopOuter = True


        # Convert the python function into a C function pointer.
        cFuncPtr = ps.StreamingReadyType(streaming_callback)

        # Fetch data from the driver in a loop, copying it out of the registered buffers and into our complete one.
        while nextSample < totalSamples and not autoStopOuter:
            wasCalledBack = False
            status["getStreamingLastestValues"] = ps.ps4000aGetStreamingLatestValues(chandle, cFuncPtr, None)
            if not wasCalledBack:
                # If we wereno't called back by the driver, this means no data is ready. Sleep for a short while before trying
                # again.
                time.sleep(0.01)



        # Find maximum ADC count value
        # handle = chandle
        # pointer to value = ctypes.byref(maxADC)
        maxADC = ctypes.c_int16()
        status["maximumValue"] = ps.ps4000aMaximumValue(chandle, ctypes.byref(maxADC))
        assert_pico_ok(status["maximumValue"])

        # Convert ADC counts data to mV
        adc2mVChAMax = adc2mV(bufferCompleteA, channel_range, maxADC)
        adc2mVChBMax = adc2mV(bufferCompleteB, channel_range, maxADC)
        adc2mVChCMax = adc2mV(bufferCompleteC, channel_range, maxADC)
        adc2mVChDMax = adc2mV(bufferCompleteD, channel_range, maxADC)
        adc2mVChEMax = adc2mV(bufferCompleteE, channel_range, maxADC)

        arrayA = np.array(adc2mVChAMax)
        arrayB = np.array(adc2mVChBMax)
        arrayC = np.array(adc2mVChCMax)
        arrayD = np.array(adc2mVChDMax)
        arrayE = np.array(adc2mVChEMax)

        # path ='D:/Amit/Codes/python/My codes/Modified streaming example/dataset_4032021/AGAIN'
        # folder_1 = (datetime.datetime.now())
        # fname = str(folder_1.strftime("%H"))+'-'+str(folder_1.strftime("%M"))+'-'+str(folder_1.strftime("%S"))
        # os.makedirs(f'{path}/{fname}')
        # np.savetxt(f"{path}/{fname}/ChannelA_orig.csv", arrayA, delimiter=",")
        # np.savetxt(f"{path}/{fname}/ChannelB_orig.csv", arrayB, delimiter=",")
        # np.savetxt(f"{path}/{fname}/ChannelC_orig.csv", arrayC, delimiter=",")
        # np.savetxt(f"{path}/{fname}/ChannelD_orig.csv", arrayD, delimiter=",")
        # np.savetxt(f"{path}/{fname}/ChannelE_orig.csv", arrayE, delimiter=",")

        channelA = arrayA
        channelB = arrayB
        channelC = arrayC
        channelD = arrayD
        channelE = arrayE

        return channelA, channelB, channelC, channelD, channelE 

    if __name__ == "__main__":
        global T

        start_time = time.time()
        # creating threads
        t1 = threading.Thread(target=getArduinoData, name='t1')
        t2 = threading.Thread(target=getHPdata, name='t2')

        # starting threads
        t1.start()
        t2.start()

        # wait until all threads finish
        t1.join()
        t2.join()
        end_time = time.time()
        T = end_time - start_time

        print('\n')
        print("Process p1 is alive: {}".format(t1.is_alive()))
        print("Process p2 is alive: {}".format(t2.is_alive()))
        print("Time Taken in threading: ", (time.thread_time())/2)
        print("Time taken by the loop: ", T)
        print("\n")
        print("Type of t1 and t2: ", type(t1))





    print("Size of Array A: ", np.size(channelA))
    print("Channel A: ", channelA)
    print('\n')
    print("Size of Array Ax: ", np.size(Ax))
    print(" Ax : ", Ax)

    # Plot data from channel A and B

    %matplotlib inline

    plt.plot(channelA)
    plt.title("Channel A from Picoscope")
    plt.show()

    plt.plot(Ax)
    plt.title("Ax from Arduino")
    plt.show()

In [ ]:
    # notes for function def stage_1(signal, winsize_zero_cross, fs, run_plot)
    # stage 1 does the following processes:
    # 1. filter use IIR bandpass filters
    # 2. estimate the zero-crossing of the first peak in two stages
    #    first of which is approximate position where first peak crosses the threshold of 10% of signal mean
    #    second stage, wherein use curvature of the signal to estimate a more-accurate estimate of zero-crossing
    def Est_ZC_stage_1(signal, winsize, fs, run_plot, plot_title, std_noise_multiplier):

        def sum_index(index_limit, reshaped_group_results_len):
            ind_count = 0
            for k in range(0, index_limit):
                ind_count = ind_count + reshaped_group_results_len[k]
            return ind_count

        def ZC_estimation(signal_h, noise_threshold, n_max_p):

            # estimate out part of the signal which is > threshold
            results = np.where(signal_h>noise_threshold, True, False)

            # itertools groupby aims to group the array in groups of True and False
            # where g is the groups and k is unique key/ value in each group
            # size/ length of the groups is given by len(list(g)) and is stored as an np.array
            group_results_len = np.array([len(list(g)) for k, g in itertools.groupby(results)])

            # only the first value of each groups (True or False) is stored in this array - will now correspond to each group
            # for which length are estimated previously
            group_results_value = np.array([list(g)[0] for k, g in itertools.groupby(results)])

            # indices & group-length wherein corresponding value in group_results_value is True
            ind_of_group_results_value_for_trues = np.array(np.where(group_results_value == True))
            group_len_counts_of_true = (group_results_len[ind_of_group_results_value_for_trues])

            # indices of top n_max maximuma of group_len_counts_of_true
            ind_n_max_group_len_counts_of_true = np.array(np.argsort(-group_len_counts_of_true))

            # manipulations to match the shape of the arrays
            group_len_counts_of_true = group_len_counts_of_true.T
            ind_n_max_group_len_counts_of_true = ind_n_max_group_len_counts_of_true.T

            # how many maxs (top-n) need to be estimated?
            n_max = int(len(group_len_counts_of_true)*(n_max_p)/100)
            if n_max <= 2:
                n_max = int(len(group_len_counts_of_true))

            ind_n_max_group_len_counts_of_true = np.array(ind_n_max_group_len_counts_of_true[0:n_max])

            # what are the values of the top-n (sorted) group-lengths of trues?
            values_maxes_in_group_results_len = np.array((group_len_counts_of_true[ind_n_max_group_len_counts_of_true])) 
            values_maxes_in_group_results_len = np.reshape(values_maxes_in_group_results_len, (n_max,1))

            # manipulations to match the shape of the arrays
            reshaped_group_results_len = np.reshape(group_results_len,(len(group_results_len),1))

            # elementwise comparison of the arrays to estimate the starting indices of the n_max groups of True:
            # comparison with the entire (true-false) list
            ind_of_max_in_group_results_len = np.array(np.where(np.in1d(reshaped_group_results_len, 
                                                                    values_maxes_in_group_results_len)))

            index_max_counts_of_true = np.zeros((len(ind_of_max_in_group_results_len), 1))
            for i in range(len(ind_of_max_in_group_results_len)):
                try:
                    index_max_counts_of_true[i] = sum_index(ind_of_max_in_group_results_len[i, 0], 
                                                            reshaped_group_results_len)+1
                except:
                    index_max_counts_of_true[i] = 0
                    break

            Q_zero_cross_index = (np.min(index_max_counts_of_true[:, 0])).astype(int)

            return Q_zero_cross_index
        def plot_ZC(signal_h, Q_zero_cross_index, plot_title, winsize):
            sig = signal_y
            index = Q_zero_cross_index
            fig, axs = plt.subplots(1)
            fig.suptitle(plot_title)
            axs.plot(sig, label = 'bandpass')
            axs.plot(np.repeat(0, len(sig)), 'k--', label = 'zero level')
            axs.plot(np.repeat(noise_threshold_1, len(sig)), 'r--', label = 'noise_threshold_1')
            axs.plot(np.repeat(noise_threshold_2, len(sig)), 'b--', label = 'noise_threshold_2')
            axs.plot(index, sig[index], 'o')
            rect = Rectangle((index, 0), winsize*2, np.max(sig)/2, linewidth=3, edgecolor='r',facecolor='none')
            axs.add_patch(rect)
            axs.legend()

        # filter first and then, normalise the signal for further processing
        signal_y = filter_bandpass(signal, fs)

        # estimate the envelope of the rectified signal using the hilbert transform
        signal_h = np.abs(hilbert(signal_y))

        # estimate threshold
        threshold = (np.mean(signal_h))
        signal_noise_h = np.where(signal_h>threshold, 0, signal_h)
        noise_threshold_1 = np.mean(signal_noise_h) + std_noise_multiplier * np.std(signal_noise_h)
        print('noise_threshold_1', noise_threshold_1)

        # first iteration 
        n_max_p = 5
        init_est_ZC_index = ZC_estimation(signal_h, noise_threshold_1, n_max_p)
        print(f'init_est_ZC_index - {plot_title}', init_est_ZC_index)

        # attempt to check the samples in the neighbourhood
        hori_check_indices = np.linspace(init_est_ZC_index-winsize, init_est_ZC_index+winsize, winsize*2, dtype = int)
        if np.all(signal_h[hori_check_indices]<10*noise_threshold_1):
            init_est_ZC_index = ZC_estimation(signal_h, noise_threshold_1*2, n_max_p)

        # attempt to check the samples in the neighbourhood by selecting a window with elevated threshold
        noise_threshold_2 = noise_threshold_1*1.25
        print('noise_threshold_2', noise_threshold_2)
        second_est_ZC_index = 0
        if signal_h[init_est_ZC_index] >= 3*noise_threshold_1:
            # second iteration 
            n_max_p = 50
            sub_signal = signal_h[init_est_ZC_index-winsize : init_est_ZC_index+winsize]
            ZC_subsignal = ZC_estimation(sub_signal, noise_threshold_2, n_max_p)
            print(f'ZC_subsignal - {plot_title}', ZC_subsignal)
            second_est_ZC_index = init_est_ZC_index + (ZC_subsignal-winsize)
            print(f'second_est_ZC_index - {plot_title}', second_est_ZC_index)

        if second_est_ZC_index!= 0:
            ZC_index = second_est_ZC_index
        else:
            ZC_index = init_est_ZC_index

        if run_plot:
            plot_ZC(signal_h, ZC_index, plot_title, winsize)

        return ZC_index

In [ ]:
    %matplotlib inline
    #format the book
    import book_format
    book_format.set_style()

In [ ]:
    from filterpy.monte_carlo import systematic_resample
    from numpy.linalg import norm
    from numpy.random import randn
    import scipy.stats

    no_of_repeatation = 5
    mu = [0,0,0,0,0]
    def run_pf1(N, iters = no_of_repeatation, sensor_std_err=0.01, 
                do_plot=True,
                xlim=(0, 10), ylim=(0, 10),zlim=(0, 10),
                initial_x=mu):
        landmarks = np.array([[1.70, 0.0, 0.75], [0.80, 8.85, 0.25], [0.0, 4.25, 0.55], [3.0, 4.50, 1.20]])
        NL = len(landmarks)
        # create gps and weights

        if initial_x is not None:
            particles = create_gaussian_particles(
                mean=initial_x, std=(1, 1, 1, np.pi/8, np.pi/8), N=N)
        else:
            particles = create_uniform_particles((0,100), (0,100), (0,100), (0, 2*np.pi), (0, 2*np.pi), N)
        weights = np.ones(N) / N


        xs = []
        robot_pos = gps_solve(distances_to_station, stations)
        pinger_coordinates_multi = gps_solve(distances_to_station, stations)
        print('robot_pos: ', robot_pos)
        for x in range(iters):                                                                #####Control_Input#####
            robot_pos= initial_x           #------robot_pos which is initial_x which is mu, is getting updated evry loop------#
            # distance from robot to each landmark
            zs = [dist_AB, dist_AC, dist_AD, dist_AE]

    #         dist_to_go = ((T**2)/2)*(math.sqrt(data[3]**2 + data[4]**2 + data[5]**2))
            Vel_x = np.trapz(data[6], dx = T)
            Vel_y = np.trapz(data[7], dx = T)
            Vel_z = np.trapz(data[8], dx = T)
            dist_to_go_x = Vel_x * T
            dist_to_go_y = Vel_y * T
            dist_to_go_z = Vel_z * T
            Net_dist_to_go = math.sqrt(dist_to_go_x**2 + dist_to_go_y**2 + dist_to_go_z**2)
            print('distnace to go in X, Y and Z : ', "{0:.3f}".format(dist_to_go_x*100), "{0:.3f}".format(dist_to_go_y*100), "{0:.3f}".format(dist_to_go_z*100))
            print('distance needs to be covered in 3D space: ',"{0:.3f}".format(Net_dist_to_go*100))
            print('\n')

            u = [data[1], data[2], Net_dist_to_go]

            predict(particles, u, std=(.02, .01, .01, .01))    
    #         correct(N, particles, pinger_coordinates_multi)                               #----Not in use----#

            # incorporate measurements
            update(particles, weights, z=zs, R=sensor_std_err, 
                   landmarks=landmarks)

            # resample if too few effective particles
            if neff(weights) < N/2:
                indexes = systematic_resample(weights)
                resample_from_index(particles, weights, indexes)
                assert np.allclose(weights, 1/N)
            mu, var = estimate(particles, weights)
            xs.append(mu)
    #         print ('Calculated coordinates of pinger: ' '\033[1m',mu)
    #         print ('\033[0m') 

        xs = np.array(xs)
        lastbestguess = mu
        if iters == no_of_repeatation:
            dateTimeObj = datetime.now()
            print("HH:SS:MS    :     ", dateTimeObj.minute, ':', dateTimeObj.second, ':', dateTimeObj.microsecond)
            print('Final calculated coordinates and headings  of pinger: ''\033[1m', mu)
            print ('\033[0m')
        return mu

    from numpy.random import seed
    seed(2)
    pinger_coordinates_multi = gps_solve(distances_to_station, stations)
    temp_array = [pinger_coordinates_multi[0], pinger_coordinates_multi[1], pinger_coordinates_multi[2], pinger_coordinates_multi[0], 0, 0]

    x = range(100)
    for n in x:
        if (n==0 or n%5==0):
            initial_x = mu = temp_array    
        mu = run_pf1(N=25000, initial_x = mu)
    print('FINAL COORDS: ', mu)

In [ ]:
# Stop the scope
# handle = chandle
status["stop"] = ps.ps4000aStop(chandle)
assert_pico_ok(status["stop"])

# Disconnect the scope
# handle = chandle
status["close"] = ps.ps4000aCloseUnit(chandle)
assert_pico_ok(status["close"])
print('Disconnected and closed')